Queremos hacer un estudio de mercado del sector del automóvil en España, cómo han evolucionado las ventas a lo largo del tiempo.

de aquí saco las matriculaciones por año: https://datosmacro.expansion.com/negocios/matriculaciones-vehiculos/espana?anio=2021

de aquí saco una API con los modelos = https://public.opendatasoft.com/explore/dataset/all-vehicles-model/api/

API para modelos por años: https://rapidapi.com/koukoulina001/api/cars21/playground/apiendpoint_23fe4099-3684-4e75-8a43-9c7a880e34ba

Aquí info sobre modelos: https://anfac.com/cifras-clave/matriculaciones-turismos-y-todoterreno/

Coches más vendidos por años: https://www.autofacil.es/industria/coches-mas-vendidos-historia-espana/178791.html

API Wallapop: http://api.wallapop.com/api/v3/general/search


In [2]:
import requests
import pandas as pd
import sys
sys.path.append("../")
import src
from src import soporte
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

In [5]:
lista_coches = ["SEAT IBIZA", "OPEL ASTRA", "VOLKSWAGEN	GOLF","TOYOTA C-HR", "CITROEN C4", "SEAT ARONA","FORD FOCUS","RENAULT MEGANE","DACIA SANDERO"]
df_final = pd.DataFrame()
df_medianos = pd.DataFrame()

for coche in lista_coches:
    #Creamos un df para cada modelo en específico con 40 observaciones para cada uno. El precio es orientativo
    df_coche = soporte.wallapop_coche(coche, 20000,precio_minimo= 10000)
    #Extraemos también aquellas unidades que más se acercan al precio mediano de su categoría
    print(f"Para el {coche}")
    df_median = soporte.coche_cerca_mediano(df_coche)

    #Concatenamos los dos dfs para tener la información guardada
    df_final = pd.concat([df_coche,df_final])
    df_medianos = pd.concat([df_median,df_medianos])





Para el SEAT IBIZA
el precio del modelo que más se acerca al precio mediano es: 13500.0
Para el OPEL ASTRA
el precio del modelo que más se acerca al precio mediano es: 12790.0
Para el VOLKSWAGEN	GOLF
el precio del modelo que más se acerca al precio mediano es: 15000.0
Para el TOYOTA C-HR
el precio del modelo que más se acerca al precio mediano es: 18081.0
Para el CITROEN C4
el precio del modelo que más se acerca al precio mediano es: 14500.0
Para el SEAT ARONA
el precio del modelo que más se acerca al precio mediano es: 16790.0
Para el FORD FOCUS
el precio del modelo que más se acerca al precio mediano es: 15890.0
Para el RENAULT MEGANE
el precio del modelo que más se acerca al precio mediano es: 14100.0
Para el DACIA SANDERO
el precio del modelo que más se acerca al precio mediano es: 12900.0


In [ ]:
# ruta_csv1 = "C:\\Users\\Víctor\\Desktop\\HACKIO\\Proyecto5\\Datos\\CSVs\\Modelos_populares_wallapop.csv"
# ruta_csv2 = "C:\\Users\\Víctor\\Desktop\\HACKIO\\Proyecto5\\Datos\\CSVs\\Modelos_mediana_populares.csv"

# df_final.to_csv(ruta_csv1, index= False, encoding= "utf-8")
# df_median.to_csv(ruta_csv2, index= False, encoding= "utf-8")